# Segmenting and Clustering Neighborhoods in Toronto
## Question 1
### importation packages

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
df_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
type(df_table)
df_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
df_table.shape

(180, 3)

In [8]:
# Borough has 77 numbers of Not assigned
df_table["Borough"].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [9]:
# let's write the same Dataframe by using not equal to operate or
df_table = df_table.query('Borough != "Not assigned"') #removing "Not assigned" from Borough
df_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
df_table.shape

(103, 3)

In [11]:
# let's reset the index
df_table = df_table.reset_index(drop=True)
df_table

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Question  2
### importation packages

In [13]:
!pip install geocoder
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [14]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal_Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)

43.648690000000045 -79.38543999999996


In [15]:
# read geospatial data file
url = 'http://cocl.us/Geospatial_data'
geotable = pd.read_csv(url)
geotable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Let's append lat & long into table
toronto = pd.merge(df_table, geotable, left_on='Postal Code', right_on='Postal Code',
                   left_index=False, right_index=False)
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [20]:
import folium
#make a map of all Toronto neighborhoods
#Get the coordinates of Toronto
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

#create a folium map of Toronto with Boroughs
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill_color='blue').add_to(map_Toronto)
map_Toronto

In [21]:
df_table

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [22]:
df_table.groupby("Borough").count()

,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Scarborough,17,17
West Toronto,6,6


In [23]:
# let us choose Downtown Toronto as Toronto being the major Canadian city and capital of the province of Ontario.
# selecting only downtown Toronto 

downtown = toronto[toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [24]:
downtown.shape

(19, 5)

In [26]:
#make a map of Downtown neighborhoods.
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal_Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

map_Downtown = folium.Map(location=[latitude, longitude], zoom_start=13
                        )
for lat, lng, neighborhood in zip(downtown['Latitude'], downtown['Longitude'], downtown['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label)
    folium.Marker([lat, lng], radius=3, popup=label, color='black', fill_color='red').add_to(map_Downtown)
map_Downtown

In [27]:
CLIENT_ID = 'FE1YPUCWNFCHK4DZADVVSUCCDZH1OET42WLT5IR4BV32OSA1'
CLIENT_SECRET = 'LKUAPABR20SZQZSKNT3MJWK2OJODBPYYARSQQGGGSZISCCVX'
VERSION = '20180605'
print("loaded")

loaded


In [29]:
# Let's explore the first neighborhood in the dataframe. If I have time, I'd like to do this again with more recent data because of the Stay at Home orders that are out there.
RegentHarbour = downtown

neighborhood_name = RegentHarbour.loc[0, 'Neighbourhood']
neighborhood_latitude = RegentHarbour.loc[0, 'Latitude']
neighborhood_longitude = RegentHarbour.loc[0, 'Longitude']
print('The latitude and longitude of {} are {} and {}.'.format(neighborhood_name,
                                                                  neighborhood_latitude, neighborhood_longitude))

The latitude and longitude of Regent Park, Harbourfront are 43.6542599 and -79.3606359.


In [30]:
# Get the top 100 venues in Regent Park and Harbourfront.
LIMIT =100
radius = 500
Top100 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
Top100

'https://api.foursquare.com/v2/venues/explore?client_id=FE1YPUCWNFCHK4DZADVVSUCCDZH1OET42WLT5IR4BV32OSA1&client_secret=LKUAPABR20SZQZSKNT3MJWK2OJODBPYYARSQQGGGSZISCCVX&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

In [32]:
results = requests.get(Top100).json()
results

{'meta': {'code': 200, 'requestId': '5f19dc4c588ea57d025f09db'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
            
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
from pandas.io.json import json_normalize
# Let's clean this up a bit and put it into a dataframe. For fun, I'm going to include the address in my df, not just category.
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# Filter the columns that we'd like in the df.
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.address']
nearby_venues = nearby_venues.loc[:, filtered_columns]

#Filter the category for each row. (I feel like this could be considered applying our category definition to the df, how we'd like it displayed.)
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#Define how we'd like to separate the columns.
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng,address
0,Roselle Desserts,Bakery,43.653447,-79.362017,362 King St E
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809,368 King St E
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008,461 Cherry St
3,Body Blitz Spa East,Spa,43.654735,-79.359874,497 King Street East
4,Impact Kitchen,Restaurant,43.656369,-79.356980,573 King St E


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [40]:
# Let's get that list of venues in a df with the lat/long for the venue.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        #api request
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        #get request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only the info we want
        venues_list.append([(name, lat, lng,
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood'],['Neighbourhood Latitude'],['Neighbourhood Longitude'],['Venue'],['Venue Latitude'],['Venue Longitude'],['Venue Category']
            
    return(nearby_venues)

In [41]:
downtown_venues = getNearbyVenues(names=downtown['Neighbourhood'], latitudes=downtown['Latitude'], longitudes=downtown['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [42]:
print(downtown_venues.shape)
downtown_venues.head()

(1243, 7)


,"(Neighbourhood,)","(Neighbourhood Latitude,)","(Neighbourhood Longitude,)","(Venue,)","(Venue Latitude,)","(Venue Longitude,)","(Venue Category,)"
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [43]:
downtown_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [44]:
#Number of venues in each Neighborhood
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [45]:
#one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood column
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

#move neighborhood to first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

print(downtown_onehot.shape)
downtown_onehot.head()

(1243, 207)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#group rows by neighborhood and find freqency of each category
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.012821,0.012821,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [47]:
#Find the top 5 venues for each neighborhood.
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1      Farmers Market  0.03
2              Bakery  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3             Plane  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.06
2       Sandwich Place  0.06
3                 Café  0.05
4  Japanese Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Restaurant  0.06
4     Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.05
3              Gay Bar  0.04
4  

In [48]:
#Let's put this in a df by the most common venues

#Start by putting our categories in order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
import numpy as np
#Create the dataframe with the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']  #As in 1st, 2nd, 3rd

#create the columns
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) #column_number+'st', column_number+'nd', column_number+'rd'
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) #4th, 5th, etc
    
#create df
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Beer Bar,Bakery,Restaurant,Farmers Market,Seafood Restaurant,Cocktail Bar,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Plane,Boat or Ferry,Coffee Shop,Harbor / Marina,Bar,Airport Gate
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Middle Eastern Restaurant,Salad Place,Japanese Restaurant,Bar,Department Store
3,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Nightclub,Candy Store,Restaurant,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Smoke Shop


In [51]:
toronto_buroughs = toronto[toronto['Borough'].astype(str).str.contains('Toronto')]
toronto_buroughs

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [57]:
# Let's get that list of venues in a df with the lat/long for the venue.
latitudes = toronto_buroughs['Latitude']
longitudes = toronto_buroughs['Longitude']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        #api request
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        #get request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only the info we want
        venues_list.append([(name, lat, lng,
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood'],['Neighbourhood Latitude'],['Neighbourhood Longitude'],['Venue'],['Venue Latitude'],['Venue Longitude'],['Venue Category']
            
    return(nearby_venues)

In [61]:
toronto_buroughs_venues = getNearbyVenues(names=toronto_buroughs['Neighbourhood'], latitudes=toronto_buroughs['Latitude'], longitudes=toronto_buroughs['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [62]:
print(toronto_buroughs_venues.shape)
toronto_buroughs_venues

(1649, 7)


,"(Neighbourhood,)","(Neighbourhood Latitude,)","(Neighbourhood Longitude,)","(Venue,)","(Venue Latitude,)","(Venue Longitude,)","(Venue Category,)"
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
1644,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1645,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1646,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1647,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [63]:
# Stupid column names
toronto_buroughs_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
toronto_buroughs_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [64]:
#How many are in each neighborhood?
toronto_buroughs_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [65]:
print('There are {} unique categories.'.format(len(toronto_buroughs_venues['Venue Category'].unique())))

There are 234 unique categories.


In [66]:
#one hot
toronto_buroughs_onehot = pd.get_dummies(toronto_buroughs_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_buroughs_onehot['Neighborhood'] = toronto_buroughs_venues['Neighborhood']

fixed_columns = [toronto_buroughs_onehot.columns[-1]] + list(toronto_buroughs_onehot.columns[:-1])
toronto_buroughs_onehot = toronto_buroughs_onehot[fixed_columns]

toronto_buroughs_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_buroughs_onehot.shape

(1649, 234)

In [68]:
toronto_buroughs_grouped = toronto_buroughs_onehot.groupby('Neighborhood').mean().reset_index()
toronto_buroughs_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.015385,0.000000,0.000000,0.015385,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.012821,0.012821,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


In [69]:
toronto_buroughs_grouped.shape

(39, 234)

In [70]:

#Top 5 venue from each neighborhood.
num_top_venues = 5

for hood in toronto_buroughs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_buroughs_grouped[toronto_buroughs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1            Café  0.03
2    Cocktail Bar  0.03
3  Farmers Market  0.03
4      Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3          Bakery  0.08
4      Restaurant  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2          Restaurant  0.05
3              Garden  0.05
4       Garden Center  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3          Boutique  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Resta

In [110]:
#Let's recreate our Top 10 dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
Tburoughs_venues_sorted = pd.DataFrame(columns=columns)
Tburoughs_venues_sorted['Neighborhood'] = toronto_buroughs_grouped['Neighborhood']

for ind in np.arange(toronto_buroughs_grouped.shape[0]):
    Tburoughs_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_buroughs_grouped.iloc[ind, :], num_top_venues)
    
Tburoughs_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Restaurant,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Breakfast Spot,Nightclub,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Japanese Restaurant,Department Store,Bar


In [112]:
from sklearn.cluster import KMeans
# Now for the good part!
#k-means clustering
kclusters = 5
tb_grouped_clustering = toronto_buroughs_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tb_grouped_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 2, 0, 2, 0, 0, 0, 0])

In [114]:
#Create a new df that includes the cluster and Top 10 for each neighborhood
#add clustering labels
Tburoughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
TB_merged = toronto_buroughs

#merge TB_merged w/ toronto to add lat/long
TB_merged = TB_merged.join(Tburoughs_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TB_merged.head()

ValueError: cannot insert Cluster Labels, already exists